**NOTE**: Please be mindful that the log messages are logged across the code box that instantiates the object, so if a log message from a previous execution of the notebook does not match up with the markdown notes, please re-run the code again and look in the first box. I don't know how to fix that, sorry!

# Submit your samples validating them against a checklist

This notebook is an extension from the first notebook; here, we will focus on another aspect of a biosamples submission: validation against a checklist.

Checklists are the name that the validation rules for the samples receive. Currently, BioSamples is mostly tailored to 2 types of checklists:
- BioSamples checklists: Accessioned as "BSDXXX"
- ENA checklists: Accessioned as `ERCXXX`. Full list, with explanation on mandatory/required fields, can be found [here](https://www.ebi.ac.uk/ena/browser/checklists)

For the first steps, we will do the same as in the previous notebook: Set up the required components, and create a sample.

In [3]:
## Import everything we need
from biobroker.authenticator import WebinAuthenticator # Biosamples uses the WebinAuthenticator
from biobroker.api import BsdApi # BioSamples Database (BSD) API
from biobroker.metadata_entity import Biosample # The metadata entity
from biobroker.input_processor import TsvInputProcessor # An input processor
from biobroker.output_processor import XlsxOutputProcessor # An output processor
import os

## Generate sample
sample_tsv = [
    ["name", "collected_at", "organism", "release"],
    ["sumple", "noon", "Homo sapiens", "2024-07-10"]         
]

writable_sample = "\n".join(["\t".join(row) for row in sample_tsv])
with open("simple_sample_sumple.tsv", "w") as f:
    f.write(writable_sample)

path = "simple_sample_sumple.tsv" # This is the file we created previously

## Set up the required entities

input_processor = TsvInputProcessor(input_data=path)
my_sample = input_processor.process(Biosample)

os.environ['API_ENVIRONMENT'] = "dev" # There are multiple ways to set up environment variables

username = "" # Your username goes here
password = "" # Your password goes here
authenticator = WebinAuthenticator(username=username, password=password)

api = BsdApi(authenticator=authenticator)

## Choosing a checklist and validating

After everything is set-up, we need to choose one of the checklist for validation. Let's say we browsed the available checklists, and since our sample is of human skin, we want to align with the GSC MIxS Human Skin checklist (https://www.ebi.ac.uk/ena/browser/view/ERC000017). 

In BioSamples, checklists are chosen by attaching the accesion of the list as metadata to the sample. Let's start by adding just the checklist and trying to submit, to see what happens:

In [4]:
my_sample[0]['checklist'] = "ERC000017"
api.submit(my_sample)

BiosamplesValidationError: Found following errors in sample validation:
	- /characteristics.project name: should have required property 'project name'
	- /characteristics.collection date: should have required property 'collection date'
	- /characteristics.geographic location (country and/or sea): should have required property 'geographic location (country and/or sea)'
	- /characteristics.geographic location (latitude): should have required property 'geographic location (latitude)'
	- /characteristics.geographic location (longitude): should have required property 'geographic location (longitude)'
	- /characteristics.broad-scale environmental context: should have required property 'broad-scale environmental context'
	- /characteristics.local environmental context: should have required property 'local environmental context'
	- /characteristics.environmental medium: should have required property 'environmental medium')

As we can see, a custom python Exception is raised when we try to do this; This is the biobroker's way of raising the checklist errors. It's complaining about a bunch of missing fields, so let's fix that!

(Please note: For the purpose of this notebook, we will do it here, but it would be much, MUCH easier to just fix the input tsv or xlsx file and load the samples again)

In [5]:
my_sample[0]['project name'] = "Your fake project"
my_sample[0]['organism'] = "Homo sapiens"
my_sample[0]['collection date'] = "2024-09-01"
my_sample[0]['geographic location (country and/or sea)'] = "Mushroom kingdom"
my_sample[0]['geographic location (latitude)'] = 1.2234
my_sample[0]['geographic location (longitude)'] = 7.21
my_sample[0]['broad-scale environmental context'] = "United Kingdom weather"
my_sample[0]['local environmental context'] = "Mostly rainy"
my_sample[0]['environmental medium'] = "Please read my plant"

api.submit(my_sample)

BiosamplesValidationError: Found following errors in sample validation:
	- geographic location (country and~1or sea)/0/text: should be equal to one of the allowed values: ["Afghanistan","Albania","Algeria","American Samoa","Andorra","Angola","Anguilla","Antarctica","Antigua and Barbuda","Arctic Ocean","Argentina","Armenia","Aruba","Ashmore and Cartier Islands","Atlantic Ocean","Australia","Austria","Azerbaijan","Bahamas","Bahrain","Baker Island","Baltic Sea","Bangladesh","Barbados","Bassas da India","Belarus","Belgium","Belize","Benin","Bermuda","Bhutan","Bolivia","Borneo","Bosnia and Herzegovina","Botswana","Bouvet Island","Brazil","British Virgin Islands","Brunei","Bulgaria","Burkina Faso","Burundi","Cambodia","Cameroon","Canada","Cape Verde","Cayman Islands","Central African Republic","Chad","Chile","China","Christmas Island","Clipperton Island","Cocos Islands","Colombia","Comoros","Cook Islands","Coral Sea Islands","Costa Rica","Cote d'Ivoire","Croatia","Cuba","Curacao","Cyprus","C

What do you mean Mushroom Kingdom is not an accepted country or sea? ugh

In [6]:
my_sample[0]['geographic location (country and/or sea)'] = "Mediterranean Sea"

new_sample = api.submit(my_sample)

And it's done!! Let's see what it looks like:

In [8]:
print(f"https://wwwdev.ebi.ac.uk/biosamples/samples/{new_sample[0]['accession']}")

https://wwwdev.ebi.ac.uk/biosamples/samples/SAMEA131421914
